In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
win = ConversationBufferWindowMemory(k=20, return_messages=True)

In [3]:
win.chat_memory.add_user_message("hi")
win.chat_memory.add_ai_message("我可以帮你什么?")
win.buffer

[HumanMessage(content='hi'), AIMessage(content='我可以帮你什么?')]

# 简单用法

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_zhipu import ChatZhipuAI
from langchain_core.output_parsers import StrOutputParser

# 定义chain
model = ChatZhipuAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "给我一个名字即可，不要输出其他。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model

In [5]:
from textlong.memory import MemoryManager, WithMemoryBinding

# 定义记忆体
memory = MemoryManager()

# 记忆绑定管理
withMemoryChain = WithMemoryBinding(chain, memory)

In [6]:
withMemoryChain.get_input_schema()

pydantic.main.RunnableWithChatHistoryInput

In [7]:
withMemoryChain.invoke({
    "input": "我姓薛，我的孩子出生时我拿着一捧百合花到妻子面前，所以我的孩子叫"},
    config={"configurable": {"session_id": "1"}}
)

AIMessage(content='薛百合。', response_metadata={'id': '8760198949855519118', 'created': 1718696527, 'token_usage': {'completion_tokens': 5, 'prompt_tokens': 38, 'total_tokens': 43}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-1eb44a23-30ad-437b-aff0-f23fa84ff1c6-0')

In [8]:
withMemoryChain.invoke({
    "input": "能换一个吗？"},
    config={"configurable": {"session_id": "1"}}
)

AIMessage(content='当然可以，那么名字可以是“林晨”。', response_metadata={'id': '8760194139491953919', 'created': 1718696530, 'token_usage': {'completion_tokens': 12, 'prompt_tokens': 49, 'total_tokens': 61}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-a019af11-821e-485e-8077-4a79fb8c5ac3-0')

In [9]:
memory.get_memory("1")

[HumanMessage(content='我姓薛，我的孩子出生时我拿着一捧百合花到妻子面前，所以我的孩子叫'),
 AIMessage(content='薛百合。', response_metadata={'id': '8760198949855519118', 'created': 1718696527, 'token_usage': {'completion_tokens': 5, 'prompt_tokens': 38, 'total_tokens': 43}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-1eb44a23-30ad-437b-aff0-f23fa84ff1c6-0'),
 HumanMessage(content='能换一个吗？'),
 AIMessage(content='当然可以，那么名字可以是“林晨”。', response_metadata={'id': '8760194139491953919', 'created': 1718696530, 'token_usage': {'completion_tokens': 12, 'prompt_tokens': 49, 'total_tokens': 61}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-a019af11-821e-485e-8077-4a79fb8c5ac3-0')]

In [10]:
memory.get_store("1")

[HumanMessage(content='我姓薛，我的孩子出生时我拿着一捧百合花到妻子面前，所以我的孩子叫'),
 AIMessage(content='薛百合。', response_metadata={'id': '8760198949855519118', 'created': 1718696527, 'token_usage': {'completion_tokens': 5, 'prompt_tokens': 38, 'total_tokens': 43}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-1eb44a23-30ad-437b-aff0-f23fa84ff1c6-0'),
 HumanMessage(content='能换一个吗？'),
 AIMessage(content='当然可以，那么名字可以是“林晨”。', response_metadata={'id': '8760194139491953919', 'created': 1718696530, 'token_usage': {'completion_tokens': 12, 'prompt_tokens': 49, 'total_tokens': 61}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-a019af11-821e-485e-8077-4a79fb8c5ac3-0')]

In [11]:
withMemoryChain.invoke({
    "input": "我姓周，我的孩子出生时我拿着一捧茉莉花到妻子面前，所以我的孩子叫"},
    config={"configurable": {"session_id": "2"}}
)

AIMessage(content='周茉。', response_metadata={'id': '8760183384893813089', 'created': 1718696532, 'token_usage': {'completion_tokens': 5, 'prompt_tokens': 38, 'total_tokens': 43}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-2a75b120-082b-4476-ac17-2a6e29b54299-0')

In [12]:
memory.get_store("2")

[HumanMessage(content='我姓周，我的孩子出生时我拿着一捧茉莉花到妻子面前，所以我的孩子叫'),
 AIMessage(content='周茉。', response_metadata={'id': '8760183384893813089', 'created': 1718696532, 'token_usage': {'completion_tokens': 5, 'prompt_tokens': 38, 'total_tokens': 43}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-2a75b120-082b-4476-ac17-2a6e29b54299-0')]

In [13]:
memory.get_memory("3")

[]

# LocalFile 持久化

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_zhipu import ChatZhipuAI
from langchain_core.output_parsers import StrOutputParser

# 定义chain
model = ChatZhipuAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个数学老师。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model | StrOutputParser()

In [15]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from textlong.memory import LocalFileStore, MemoryManager, WithMemoryBinding, create_session_id

# 定义记忆体
memory = MemoryManager(
    lambda session_id: LocalFileStore(session_id),
    memory = ConversationBufferWindowMemory(return_messages=True, k=2)
)

# 记忆绑定管理
withMemoryChain = WithMemoryBinding(chain, memory)

In [16]:
session_id = create_session_id()
session_id

'2024-06-18-154217-6847-default_user'

In [17]:
# 调用
withMemoryChain.invoke(
    {"input": "方程式是什么意思?"},
    config={"configurable": {"session_id": session_id}}
)

'方程式（Equation）在数学中是指含有一个或多个未知数（通常用字母表示）以及等号的数学表达式。方程式用来表示两个表达式的值相等，通常包含算术运算符（如加法、减法、乘法和除法）以及可能的其他数学函数。方程式的目的是找到使得等式成立的未知数的值。\n\n方程式的一般形式如下：\n\n\\[ F(x) = G(x) \\]\n\n其中 \\( F(x) \\) 和 \\( G(x) \\) 是数学表达式，\\( x \\) 是未知数。要解方程就是找到所有满足等式的 \\( x \\) 的值。\n\n方程可以根据未知数的个数、方程的度数以及所涉及的运算类型等因素进行分类。例如，线性方程是最简单的方程类型之一，它的最高次数为一次。而二次方程则包含了二次项（\\( x^2 \\)），例如 \\( ax^2 + bx + c = 0 \\)。还有更复杂的多项式方程、指数方程、对数方程等等。'

In [9]:
# 调用
withMemoryChain.invoke(
    {"input": "你可以用小学生二年级语言给我解释吗?"},
    config={"configurable": {"session_id": session_id}}
)

'当然可以！想象一下你有一堆糖果，左边有一些，右边也有一些，然后你发现左边的糖果和右边的糖果数量是一样的。在数学中，我们用方程式来表示这种情况。\n\n方程式就像是一个小秘密，它告诉你两边的东西是一样多的。我们用这个符号“=”来表示两边是相等的。比如，这个方程式：\n\n\\[ 3 + 2 = 5 \\]\n\n就像是说，左边的3个糖果加上2个糖果，和右边的5个糖果一样多。\n\n如果我们有一个未知数，比如用“x”来表示，那么方程式可能是这样的：\n\n\\[ 3 + x = 5 \\]\n\n这个方程式就像是在问：“左边有3个糖果，再加上多少个糖果（我们不知道的x个），才能和右边的5个糖果一样多？”\n\n然后我们就可以找出答案，看看x是多少，让方程式成立。在这个例子里，x就是2，因为3加上2等于5。\n\n方程式就是这样一个找出未知数是多少的游戏，让等号两边的东西一样多。'

In [12]:
# 调用
withMemoryChain.invoke(
    {"input": "那谢谢哦"},
    config={"configurable": {"session_id": session_id}}
)

'不客气！如果你还有其他问题或者需要帮助，随时欢迎来问。希望你学习愉快！'

In [13]:
for chunk in withMemoryChain.stream(
    {"input": "我最初的问题是什么来着?"},
    config={"configurable": {"session_id": session_id}},
):
    print(chunk, end="|", flush=True)

你|最初|的问题是|：“|你|是一个|数学|老师|。”| 你|可能|是在|设定|一个|场景|，|假设|我是一个|数学|老师|，|以便|我可以|以|数学|老师的|身份|帮助你|解答|数学|问题|。|如果你|有|其他|数学|相关|的问题|或者|需要|进一步|的帮助|，|请|随时|告诉我|！||

In [10]:
memory.shorterm_messages(session_id)

[HumanMessage(content='那谢谢哦'),
 AIMessage(content='不客气！如果你有其他问题或者需要帮助，随时欢迎来问。祝你学习愉快！'),
 HumanMessage(content='我最初的问题是什么来着?'),
 AIMessage(content='你最初的问题是：“你是一个数学老师。” 你在设定一个角色，好像我是你的数学老师，准备好回答你的数学问题或者解释数学概念。如果你有其他数学相关的问题或者需要进一步的解释，请随时告诉我。')]

In [11]:
memory.longterm_messages(session_id)

[HumanMessage(content='方程式是什么意思?'),
 AIMessage(content='方程式是一个数学表达式，它表示两个表达式的值相等，通常包含一个或多个未知数，用等号“=”连接。在方程式中，未知数通常用字母（如x、y）来表示，目的是要找出这些未知数的值，使得等号两边的表达式相等。\n\n方程式的一般形式如下：\n\n\\[ A(x) = B(x) \\]\n\n其中，\\( A(x) \\) 和 \\( B(x) \\) 是数学表达式，可以包含数字、变量和运算符。方程式的解是指使得等号两边相等的未知数的值。\n\n例如，下面是一个简单的线性方程：\n\n\\[ 2x + 3 = 7 \\]\n\n这个方程的解是 \\( x = 2 \\)，因为当 \\( x \\) 等于2时，等号两边的值都是7。'),
 HumanMessage(content='你可以用小学生二年级语言给我解释吗?'),
 AIMessage(content='当然可以！想象一下你有一袋苹果，如果我把苹果分给你和其他小朋友，然后我告诉你总共分了多少个苹果，你需要找出每个小朋友分到了几个苹果。\n\n方程式就像是一个小谜题，它说的是：“这些苹果分成几份后，每份有几个苹果？”比如，如果我们这样写：\n\n\\[ 2x + 3 = 7 \\]\n\n这个方程式可以这样说：“如果你把每份苹果的数量叫做‘x’，然后你拿了两份，再加上三个苹果，一共是7个苹果。那么，每份有几个苹果呢？”\n\n在这个例子中，我们要找出“x”是多少。我们可以用简单的步骤来解决这个谜题：\n\n1. 先从7个苹果中减去那额外的3个苹果，因为那3个苹果不是我们要找的“每份”的苹果。\n   \\( 7 - 3 = 4 \\)\n\n2. 现在我们知道，剩下的4个苹果是两份的总量。所以我们只需要把4个苹果平均分给两份。\n   \\( 4 ÷ 2 = 2 \\)\n\n所以，每份苹果就是2个。这就是“x”的值，也就是我们的答案。每个小朋友分到了2个苹果。'),
 HumanMessage(content='那谢谢哦'),
 AIMessage(content='不客气！如果你有其他问题或者需要帮助，随时欢迎来问。祝你学习愉快！'),
 HumanMessage(content='我最初的问题是什么来着?'

# redis 持久化

```shell
pip install -U langchain-community redis
```

创建窗口管理器时要指定redis的工厂函数，其他与上面的例子相同。

In [ ]:
from textlong import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory

window = ConversationBufferWindowMemory(
    return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(
    lambda session_id: RedisChatMessageHistory(
        session_id, url="redis://localhost:6379"
    ),
    shorterm_memory = window
)